In [1]:
spark

Waiting for a Spark session to start...

Spark application_1540458187951_69601: Some(http://gw02.itversity.com:4041)

In [2]:
spark.sparkContext.getConf.getAll.foreach(x=>if(x._2.contains("/proxy/")){println(x._2)})

Waiting for a Spark session to start...

http://rm01.itversity.com:19288/proxy/application_1540458187951_69601


In [3]:
import scala.collection.JavaConversions._

In [4]:
val os_name = System.getProperty("os.name")
val hdfs_home = "/user/" + System.getProperty("user.home").split("/")(2)

os_name = Linux
hdfs_home = /user/kranthidr


/user/kranthidr

In [5]:
val path = hdfs_home+"/dataSets/wikidump-02-sept-2018.xml"

path = /user/kranthidr/dataSets/wikidump-02-sept-2018.xml


/user/kranthidr/dataSets/wikidump-02-sept-2018.xml

In [6]:
import com.cloudera.datascience.lsa._
import com.cloudera.datascience.lsa.AssembleDocumentTermMatrix
import com.cloudera.datascience.lsa.LSAQueryEngine
import com.cloudera.datascience.lsa.RunLSA._

In [7]:
import breeze.linalg.{DenseMatrix => BDenseMatrix, SparseVector => BSparseVector}

import org.apache.spark.mllib.linalg.{Matrices, Matrix, SingularValueDecomposition, Vectors, Vector => MLLibVector}
import org.apache.spark.ml.linalg.{Vector => MLVector}
import org.apache.spark.mllib.linalg.distributed.RowMatrix
import org.apache.spark.serializer.KryoSerializer
import org.apache.spark.sql.{Dataset, SparkSession}

import scala.collection.Map
import scala.collection.mutable.ArrayBuffer

import breeze.linalg.{DenseMatrix=>BDenseMatrix, SparseVector=>BSparseVector}
import org.apache.spark.mllib.linalg.{Matrices, Matrix, SingularValueDecomposition, Vectors, Vector=>MLLibVector}
import org.apache.spark.ml.linalg.{Vector=>MLVector}


In [ ]:
val k =  100
val numTerms = 20000

In [ ]:
val assembleMatrix = new AssembleDocumentTermMatrix(spark)
import assembleMatrix._

In [ ]:
val docTexts: Dataset[(String, String)] = parseWikipediaDump(path)

In [ ]:
docTexts.cache()

In [ ]:
val docTexts1 = docTexts.sample(false, 0.001) //50 Docs we get approx

In [ ]:
docTexts1.count()

In [ ]:
val (docTermMatrix, termIds, docIds, termIdfs) = documentTermMatrix(docTexts1, "stopwords.txt", numTerms)

In [ ]:
docTexts.unpersist()

In [ ]:
docTermMatrix.cache()

In [ ]:
 val vecRdd = docTermMatrix.select("tfidfVec").rdd.map { row =>
      Vectors.fromML(row.getAs[MLVector]("tfidfVec"))
}

In [ ]:
docTermMatrix.unpersist()

In [ ]:
vecRdd.cache()

In [ ]:
    val mat = new RowMatrix(vecRdd)

In [ ]:
    val svd = mat.computeSVD(k, computeU=true)

In [ ]:
    println("Singular values: " + svd.s)

In [ ]:
    val topConceptTerms = topTermsInTopConcepts(svd, 10, 10, termIds)

In [ ]:
    val topConceptDocs = topDocsInTopConcepts(svd, 10, 10, docIds)

In [ ]:
    for ((terms, docs) <- topConceptTerms.zip(topConceptDocs)) {
      println("Concept terms: " + terms.map(_._1).mkString(", "))
      println("Concept docs: " + docs.map(_._1).mkString(", "))
      println()
    }

In [ ]:
    val queryEngine = new LSAQueryEngine(svd, termIds, docIds, termIdfs)

In [ ]:
    queryEngine.printTopTermsForTerm("algorithm")

In [ ]:
    queryEngine.printTopTermsForTerm("kempegowda")

In [ ]:
    queryEngine.printTopTermsForTerm("malleshwaram")

In [ ]:
    queryEngine.printTopDocsForTerm("fir")

In [ ]:
    queryEngine.printTopDocsForTerm("graph")

In [ ]:
    queryEngine.printTopDocsForDoc("BAR domain")

In [ ]:
    queryEngine.printTopDocsForDoc("List of BMTC routes")

In [ ]:
    queryEngine.printTopDocsForDoc("ATP-binding cassette transporter")

In [ ]:
    queryEngine.printTopDocsForTermQuery(Seq("nagar", "circle"))